In [1]:
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import time 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.regularizers import l2

In [2]:
def showHsv(img):    
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    plt.imshow(img)
    plt.show()

In [92]:
img_in = cv2.imread('pic.jpg')
hsv = cv2.cvtColor(img_in, cv2.COLOR_BGR2HSV)
print(hsv.shape)

h,s,v = cv2.split(hsv)
#print('s',s.shape)
s = (np.random.random(s.shape) * 255).astype('uint8')
h = (np.random.random(s.shape) * 255).astype('uint8')
#s = np.maximum(0, v)
#print(s)
hsv = cv2.merge([h,s,v])
print('image with its value untouched, but saturation and hue randomized:')
#showHsv(hsv)





(1008, 1600, 3)
image with its value untouched, but saturation and hue randomized:


In [ ]:
for i in range(3):
    pic = hsv[:,:,i]
    plt.imshow(pic, cmap='gray')
    plt.show()

In [98]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [99]:
n = 10000
X = np.arange(n).reshape(n,1)
X = np.hstack((np.ones(n).reshape(n,1), X))
print(X.shape)
y = np.random.randn(n)*50 + X[:, 1]
#y = y.reshape(n,1)
print(y.shape, X[:,1].shape)
#print(y)
#plt.plot(X[:,1], y)
yval = y
X = np.hstack((X, yval[:,None]))
# labels: 1 if above line, 0  below
y = y > X[:,1]
y = y.reshape(n,1)
y = np.hstack((y, ~y )).astype('uint8')
print(y.shape)
colors = ['red' if i  else 'green' for i in y[:,0]]
#plt.scatter(X[:,1], yval, color=colors)
#plt.show()
#print(y)
#print(X)


(10000, 2)
(10000,) (10000,)
(10000, 2)


In [107]:
def createTestModel(layers):
    model = Sequential()
    #print(X.shape[1])
    model.add(Dense(layers[0], input_dim=X.shape[1], kernel_regularizer=l2(0.001)) ) 
    model.add(Activation('relu'))
    for l in layers[1:]:
        model.add(Dense(l, kernel_regularizer=l2(0.001)))
        model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                 metrics=['accuracy'])
    return model

In [108]:
currTime = time.strftime("%d-%m-%Y-%H-%M-%S", time.localtime())
layers = [5,5]
NAME = f"test-tensorboard-{ currTime }-layers-" + "x".join(str(l) for l in layers)
print(NAME)

model = createTestModel(layers)
tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=[tensorboard])

test-tensorboard-10-03-2019-21-08-58-layers-5x5
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 175us/step - loss: 6.1578 - acc: 0.5942 - val_loss: 4.1674 - val_acc: 0.6843
Epoch 2/10
8000/8000 [==============================] - 1s 101us/step - loss: 3.5986 - acc: 0.6674 - val_loss: 2.5547 - val_acc: 0.4908
Epoch 3/10
8000/8000 [==============================] - 1s 120us/step - loss: 0.5101 - acc: 0.7586 - val_loss: 0.2555 - val_acc: 0.9100
Epoch 4/10
8000/8000 [==============================] - 1s 118us/step - loss: 0.2129 - acc: 0.9332 - val_loss: 0.2213 - val_acc: 0.9305
Epoch 5/10
8000/8000 [==============================] - 1s 117us/step - loss: 0.0925 - acc: 0.9715 - val_loss: 0.0707 - val_acc: 0.9758
Epoch 6/10
8000/8000 [==============================] - 1s 110us/step - loss: 0.0992 - acc: 0.9677 - val_loss: 0.0640 - val_acc: 0.9818
Epoch 7/10
8000/8000 [==============================] - 1s 115us/step - loss: 0.0964 - a

In [83]:
pred = model.predict_classes(X).reshape((-1,))
pred = 1 - pred
#print(pred)
evaluation = model.evaluate(X, y)
print(f'loss:{evaluation[0]}\naccuracy:{evaluation[1]}')
labels = y[:,0]
#print(labels.shape, labels)
incorrects = np.nonzero(np.not_equal(pred, labels) )
print('num incorrect', len(incorrects[0]))
#print('incorr:',incorrects)
print('\nwrong pred:', pred[incorrects])


1000/1000 [==============================] - 0s 35us/step
loss:0.12557350927591324
accuracy:0.951
num incorrect 48

wrong pred: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1]
